In [85]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from queue import Queue
from datetime import datetime, timedelta
import random

In [86]:
class ChartDrawer:
    color = (0, 255, 0)
    scale_color = (255, 0, 0)
    thickness = 2

    def __init__(self, path_in, path_out, **kwargs):
        self.path_in = path_in
        self.path_out = path_out
        self.queue = Queue(64)
        self.start = None
        self.last_y = 90

        for attr, value in kwargs:
            setattr(self, attr, value)

    def update_chart(self):
        now = datetime.now()
        if self.start is None:
            self.start = now
            return

        delta_time = now - self.start
        self.last_y += random.choice([-3, 3])

        if self.last_y < 0:
            self.last_y = 0
        if self.last_y > 160:
            self.last_y = 160

        if self.queue.full():
            self.queue.get()

        self.queue.put((delta_time.seconds, self.last_y))

    def draw_chart(self, frame):
            #Рамка
            cv2.rectangle(
                frame,
                (0, 0),
                (320, 180),
                self.color,
                thickness=1
            )
            #Шкала
            cv2.line(
                frame,
                (0, 170),
                (319, 170),
                self.scale_color,
                thickness=1
            )
            point_one = None
            x = 0
            second = 0
            for data in self.queue.queue:
                x += 5
                point_two = (x, data[1])
                if point_one is not None:
                    cv2.line(
                        frame,
                        point_one,
                        point_two,
                        self.color,
                        self.thickness
                    )

                    if second != data[0]:
                        second = data[0]
                        cv2.line(
                            frame,
                            (x, 160),
                            (x, 170),
                            self.scale_color,
                            thickness=1
                        )
                        cv2.putText(
                            frame,
                            str(second),
                            (x, 170),
                            cv2.FONT_HERSHEY_PLAIN,
                            1,# Размер к базе
                            self.scale_color,
                            1 # обводка
                        )
                point_one = point_two

    def run(self):
        cap = cv2.VideoCapture(self.path_in)
        if not cap.isOpened():
            print("Cannot open")
            exit()
        else:
            print('ok')

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(self.path_out, fourcc, 20.0, (1280,  720))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Готово ...")
                break

            self.update_chart()
            self.draw_chart(frame)
            # cv2.polylines(
            #     img=frame,
            #     pts=[self.chart],
            #     isClosed=True,
            #     color=self.color,
            #     thickness=self.thickness
            # )
            out.write(frame)

        cap.release()
        out.release()
        cv2.destroyAllWindows()

In [87]:
path_in = 'videos/video_1.mp4'
path_out = 'videos/output9.avi'

drawer = ChartDrawer(path_in, path_out)
drawer.run()

ok
Готово ...
